# Animations

So far in this tutorial we've dealt with only one time step at a time. However, we also discussed that one of the main benefits of geostationary data is its temporal resolution. It can provide us with many images of the same location quickly. Let's explore this property by making a MPEG4 video of GOES-16 ABI Mesoscale data. Due to time constraints and the time it takes to do this processing we'll be using 1-minute data only over one hour (2018-05-11 21:00Z to 2018-05-11 22:00Z).

In [ ]:
%run ../init_notebook.py
from glob import glob
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_meso/*s201813121*.nc')
len(filenames)

Normally if we wanted to operate on 960 input files (60 time steps for ABI), we'd have to sort every 16 files and then create `Scene` objects for each one. Satpy provides a helper class called the `MultiScene` to help with this and other operations that you might want to do on multiple `Scene` objects at once. One way to create a `MultiScene` is to use its `from_files` class method. This will sort the files based on time and other properties to make sure that each time step has its own Scene and can be operated on as possible.

One thing to note is that to improve performance the `MultiScene` operates on "generators" internally. Generators can be a complex subject, but basically this means that, similar to dask arrays, we don't perform operations on each `Scene` when requested. Instead we build up a series of operations to apply and do them when needed. For this reason, it is suggested that MultiScenes not be used for "exploring" data where you would normally access many properties on a Scene.

Let's create our `MultiScene` and go through some basic operations.

In [ ]:
from satpy import MultiScene
mscn = MultiScene.from_files(filenames, reader='abi_l1b')

Like a normal `Scene`, the `MultiScene` has a `load` method. Let's load our RGB from the previous lesson.

In [ ]:
mscn.load(['my_rgb'])

From our experience working with an individual `Scene` earlier, we know that our RGB requires resampling. We can call the `MultiScene.resample` method to receive a new `MultiScene` object.

In [ ]:
new_mscn = mscn.resample(resampler='native')

Finally, we can save the series of images we've loaded and resampled as an MPEG4 video. We'll specify a filename and the number of frames per second (FPS) we want the video to have. Under the hood this method uses the `imageio` python library and `ffmpeg` to make videos or GIFs out of our series of images.

Note: This may take a couple minutes for 60 frames depending on your available hardware.

In [ ]:
%%time
new_mscn.save_animation('{name}_{start_time:%Y%m%d_%H%M%S}.mp4', fps=12)

To view the movie file, navigate to the `notebooks` directory with your systems file browser and open the `.mp4` video with an application capable of playing it (Quicktime, VLC, etc).

# Stacking

The `MultiScene` can also help visualize multiple orbits of a polar-orbiter satellite. Let's take a simple example with two VIIRS passes, one from NOAA-20 and one from Suomi-NPP, and stack them on top of each other. The `from_files` method used here detects that we have NOAA-20 and Suomi-NPP data and will group the files by platform. We end up with two total Scenes to operate on.

Due to the time it takes to resample this data we will only be operating on two granules from each orbit.

In [ ]:
filenames = glob('../data/viirs_sdr/20180511_texas_fire_viirs_sdr/*j01*t203[13]*.h5') + glob('../data/viirs_sdr/20180511_texas_fire_viirs_sdr/*npp*.h5')
len(filenames)

In [ ]:
mscn = MultiScene.from_files(filenames, reader='viirs_sdr')
mscn.load(['I03'])

We'll need to resample the data to get a gridded area that can show both orbits we've selected. We'll use pyresample's `create_area_def` function like we did in the resampling lesson.

In [ ]:
from pyresample import create_area_def
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -95, 'lat_0': 25, 'lat_1': 35},
                          width=3000, height=3000,
                          area_extent=[-110, 20, -85, 50], units='degrees')
my_area

In [ ]:
new_mscn = mscn.resample(my_area)

Now we can use the `MultiScene.blend` method to combine the Scenes together in to one new Scene. By default this method will "stack" each orbit on top of the next.

In [ ]:
blended_scn = new_mscn.blend()

Now we have a normal `Scene` object that we're used to working with. Let's plot it.

Note: This can take a couple minutes to finish.

In [ ]:
%%time
%matplotlib notebook

from dask.diagnostics import ProgressBar
with ProgressBar():
    blended_scn['I03'].plot.imshow()

With more data and more time we can use this same code to combine many orbits or make a global composite of polar-orbiter data.